In [1]:
import os
import json
import torch
import httpx
from IPython.display import Markdown, display

In [ ]:
HF_API_TOKEN = ""
MODEL_URL = "https://api-inference.huggingface.co/models/Qwen/Qwen2.5-0.5B-Instruct"

In [3]:
BASE_URL = "https://router.huggingface.co/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {HF_API_TOKEN}",
    "Content-Type": "application/json"
}

model_name = "Qwen/Qwen3-0.6B"


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [6]:
def run_qwen_chat(messages, max_new_tokens: int = 512) -> str:
    """
    Запускает Qwen3 в чат-режиме БЕЗ thinking.
    Возвращает только финальный текст ответа (строку).
    """
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=True  # выключаем думающий режим ради скорости
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            temperature=0
        )

    output_ids = generated_ids[0][len(model_inputs.input_ids[0]):]
    content = tokenizer.decode(output_ids, skip_special_tokens=True).strip()
    return content

In [7]:
def extract_json(text: str):
    """
    Пытаемся вытащить первый корректный JSON из строки.
    На случай, если модель всё равно добавит лишний текст.
    """
    if not text:
        return None

    candidates = re.findall(r'\{[\s\S]*\}', text)
    for c in candidates:
        try:
            return json.loads(c)
        except Exception:
            continue
    return None

In [8]:
import re

def fix_category_by_rules(letter_text: str, llm_category: str) -> str:
    text = letter_text.lower()

    # 1. Жалоба/претензия
    if re.search(r"\bжалоб[аи]\b|\bпретензи[яи]\b|\bнедовольств[оа]\b|\bнарушени[ея]\b", text):
        return "complaint"

    # 2. Регулятор
    if re.search(r"банк россии|центральн[ыйого]+ банк|на основании .* федеральн[оого] закона|указани[ея] банк[а]? россии", text):
        return "regulator"

    # 3. Согласование
    if re.search(r"просим согласовать|на согласование|просим утвердить|утвержден[ие]|согласовани[ея]", text):
        return "approval"

    # 4. Партнёрское
    if re.search(r"предлагаем сотрудничество|партнёрск|партнерск|совместн(ый|ое|ые) проект|коммерческо[ея] предложени[ея]", text):
        return "partnership"

    # 5. Уведомление
    if re.search(r"настоящим уведомляем|сообщаем, что|доводим до вашего сведения", text):
        # если LLM не сказала complaint/regulator/approval, можно форсить notification
        if llm_category not in ("complaint", "regulator", "approval"):
            return "notification"

    # 6. По умолчанию вернём то, что сказала модель
    return llm_category


In [9]:
def preprocess_letter(text: str) -> str:
    """
    Убираем типичные приветствия/подписи, чтобы модель не залипала на них.
    Очень простая эвристика — для хакатона достаточно.
    """
    lines = [l.strip() for l in text.splitlines() if l.strip()]

    # уберём первые строки, если это приветствия
    GREETINGS = (
        "уважаемые",
        "здравствуйте",
        "добрый день",
        "добрый вечер",
        "доброе утро",
    )
    while lines and any(lines[0].lower().startswith(g) for g in GREETINGS):
        lines.pop(0)

    # уберём последние строки, если это подписи
    SIGNOFF = (
        "с уважением",
        "уважением",
        "искренне ваш",
        "искренне ваша",
        "best regards",
        "kind regards",
    )
    while lines and any(lines[-1].lower().startswith(s) for s in SIGNOFF):
        lines.pop()

    return "\n".join(lines)


In [10]:
def analyze_letter(letter_text: str) -> dict | None:
    """
    Анализ входящего письма:
    - category (одна из 6 категорий)
    - deadline_text
    - contract_number
    - laws
    - summary
    Возвращает dict или None, если JSON не распарсился.
    """

    system_prompt = '''
Ты — интеллектуальный помощник сотрудника банка.
Твоя задача — проанализировать входящее письмо и вернуть СТРОГИЙ JSON по указанному шаблону.

Требования к формату ответа:
- Верни ТОЛЬКО JSON, без текста до или после.
- Ключи JSON (category, intent, deadline_text, emotion, tone_of_voice и др.) — ТОЛЬКО на английском языке.
- Значения всех полей — ТОЛЬКО на русском языке.
- Не используй английский язык в значениях.
- Не копируй текст письма дословно.
- Не включай приветствия («Уважаемые коллеги») и подписи («С уважением») в intent или summary.
- Делай краткие переформулировки от третьего лица.
- Определи эмоциональный тон и степень формальности по содержанию письма.

Категория письма (значение поля "category") — строго одна из следующих:

1) "info_request"
   Запрос информации или документов: предоставление справок, выписок, подтверждающих документов, разъяснений.

2) "complaint"
   Жалоба или претензия: недовольство качеством услуг, ошибками операций, нарушением условий, просьба устранить проблему.

3) "regulator"
   Регуляторный запрос: письмо от надзорного органа или с упоминанием нормативных требований, законов, статей, указаний.

4) "partnership"
   Партнёрское предложение: коммерческие предложения, инициативы о сотрудничестве, предложение совместных проектов или услуг.

5) "approval"
   Запрос на согласование: просьба утвердить документ, лимит, изменения договора, условия сделки.

6) "notification"
   Уведомление или информирование: сообщения без требования действий (смена реквизитов, графиков, технические уведомления).

Эмоциональный тон письма (“emotion”) — одно из значений:
- «негативное»
- «нейтральное»
- «позитивное»
- «раздражение»
- «недовольство»
- «срочность / давление»
- «спокойный тон»
- «благодарность»

Степень формальности (“tone_of_voice”) — одно из значений:
- «формально»
- «полуформально»
- «неформально»

Что нужно извлечь:

- "category" — одна строка из списка выше (на английском).
  НЕ копируй письмо. Сделай краткое, нейтральное переформулирование.
- "deadline_text" — срок ответа из письма (например, «в течение 3 рабочих дней»), либо "" если не указан.
- "contract_number" — номер договора/счёта/контракта, если есть, иначе "".
- "laws" — список строк с упоминаниями нормативных актов (например: "ФЗ-115", "Указание Банка России №...").
- "emotion" — определённая эмоция письма.
- "tone_of_voice" — степень формальности письма.
- "summary" — краткое резюме письма (1–3 предложения на только на русском), без приветствий и без дословного копирования.

Верни СТРОГО JSON такого вида:

{
  "category": "",
  "deadline_text": "",
  "contract_number": "",
  "laws": [],
  "emotion": "",
  "tone_of_voice": "",
  "summary": ""
}

    '''.strip()



    cleaned_text = preprocess_letter(letter_text)

    user_prompt = (
        f'Вот текст письма (без приветствия и подписи):\n""" {cleaned_text} """\n\n'
        "Верни только JSON по указанному шаблону."
    )

    messages = [
        {"role": "user",  "content": user_prompt},
        {"role": "system", "content": system_prompt},
    ]

    content = run_qwen_chat(messages, max_new_tokens=32768)

    print("=== RAW CONTENT ===")
    print(content)

    data = extract_json(content)
    if data is None:
        print("❌ Не удалось распарсить JSON из ответа модели")
        return None

    # Нормализация значений
    data["category"] = (data.get("category") or "").strip()
    # data["intent"] = (data.get("intent") or "").strip()
    data["deadline_text"] = (data.get("deadline_text") or "").strip()
    data["contract_number"] = (data.get("contract_number") or "").strip()
    data["emotion"] = (data.get("emotion") or "").strip()
    data["tone_of_voice"] = (data.get("tone_of_voice") or "").strip()
    data["laws"] = data.get("laws") or []
    data["summary"] = (data.get("summary") or "").strip()

    # Корректируем категорию по правилам
    data["category"] = fix_category_by_rules(letter_text, data["category"])

    return data

In [11]:
sample_letter = """
Настоящим уведомляем о грубом нарушении условий договора №БС-1456 от 15.03.2023: средства с нашего счёта были списаны без предварительного уведомления.
Требуем немедленного разъяснения и возврата средств в течение 3 рабочих дней.

"""

result = analyze_letter(sample_letter)
print("\n=== PARSED RESULT ===")
print(json.dumps(result, ensure_ascii=False, indent=2))


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== RAW CONTENT ===
<think>
Okay, let's start by analyzing the user's request. They provided a message from a bank employee and want a JSON response based on the specified structure. The key points to extract are the category, deadline, contract number, laws, emotion, tone, and summary.

First, the message is a complaint about a mistake in the contract number. The user mentioned that the contract number was not properly acknowledged, leading to a mistake. The deadline is 3 working days, and the emotion is negative. The tone is formal as it's a regulatory notice.

For the category, since it's a complaint about a mistake in the contract, the correct category is "complaint". The contract number is "BС-1456". The deadline is "в течение 3 рабочих дней". The laws mentioned are the contract number, so I'll include that. The emotion is negative, and the tone is formal. The summary should capture the main points without being too verbose.

I need to make sure all fields are in the correct forma

In [12]:
sample_letter = """
Настоящим обращаемся к Банку с просьбой оказать благотворительную поддержку в реализации социального проекта «Дети — наше будущее», инициированного Фондом поддержки детей-сирот и детей, оставшихся без попечения родителей «Светлый путь».

Проект направлен на обеспечение детей из детских домов в регионах присутствия Банка (Москва, Санкт-Петербург, Екатеринбург, Казань) современными учебными материалами, техникой (планшеты, ноутбуки), а также организацией психологических и творческих программ.

В 2024 году проект охватил 1 200 детей, в 2025 году планируется расширение до 2 000 детей.

Мы просим рассмотреть возможность:

Выделения финансовой помощи в размере до 2 500 000 рублей на закупку оборудования и материалов;
Организации волонтёрских дней сотрудников Банка в детских домах;
Размещения информации о проекте на сайте и в соцсетях Банка (по согласованию).
Прилагаем:
Устав Фонда и свидетельство о регистрации (№123456 от 10.02.2020);
Отчёт о результатах 2024 года;
Бюджет проекта на 2025 год.
Прошу рассмотреть возможность поддержки и направить ответ в течение 15 рабочих дней до 15.12.2025 включительно.
"""

result = analyze_letter(sample_letter)
print("\n=== PARSED RESULT ===")
print(json.dumps(result, ensure_ascii=False, indent=2))


=== RAW CONTENT ===
<think>
Okay, let's start by understanding the user's request. They provided a letter from a bank to a fund about a social project and want it converted into a JSON structure. The key points are to extract specific fields and format them correctly.

First, I need to identify the category. The letter is a request for financial support, so "info_request" makes sense. The deadline is mentioned as "в течение 15 рабочих дней до 15.12.2025", which translates to "deadline_text": "в течение 15 рабочих дней до 15.12.2025". 

Next, the contract number is provided as "Устав Фонда и свидетельство о регистрации (№123456 от 10.02.2020)". Since the user wants the contract number in the JSON, I'll include it as is. 

The laws mentioned are "ФЗ-115" and "Указание Банка России №...". These are legal references, so they should be added to the "laws" array. 

The emotion here is positive, as it's a request for support. The tone_of_voice is formal, as it's a regulatory request. 

The su

In [13]:
sample_letter = """
Дочерняя организация "Рога и копыта" планирует проведение офлайн-мероприятия для корпоративных клиентов Банка 28 ноября 2025 года в Москве, площадка — конференц-зал бизнес-центра «Горизонт». Цель мероприятия — повышение лояльности и демонстрация интегрированных решений группы компаний. В программе: выступления экспертов, кейсы клиентов, нетворкинг. Направляем на согласование:
Финальная программа мероприятия (в приложении)
Список приглашённых (все — действующие клиенты Банка)
Тезисы докладов (с упоминанием бренда Банка)
Проект коммуникационного письма для рассылки клиентам
Прошу согласовать организацию и проведение мероприятия, а также использование корпоративного бренда Банка в материалах до 23 ноября 2025 года.

"""

result = analyze_letter(sample_letter)
print("\n=== PARSED RESULT ===")
print(json.dumps(result, ensure_ascii=False, indent=2))


=== RAW CONTENT ===
<think>
Okay, let's start by understanding the user's request. They provided a message from a bank's off-site event planning and want me to extract specific fields into a JSON format according to the given schema. 

First, I need to parse the message. The original text mentions the event date, location, purpose, program details, and the deadline for agreement. The key points are the deadline (November 23, 2025), the category as a request for agreement, and the other details.

Looking at the category, the message is a request to agree on the event, so "approval" makes sense. The deadline is "в течение 3 рабочих дней" which translates to "within 3 working days." The contract number isn't provided, so I'll leave that blank. The laws mentioned are the relevant regulations, but the message doesn't specify them, so I'll omit them. The emotion is neutral since it's a formal request. The tone is formal as it's a business communication.

Now, I need to structure this into th

In [14]:
sample_letter = """
Уважаемый Председатель Банка,

Настоящим уведомляем вас об изменении методики расчёта показателя «Доля просроченной задолженности по кредитам физических лиц» (далее — Показатель), применяемой при формировании отчётности в рамках Формы №110 «Отчёт о качестве кредитного портфеля».

Согласно Указанию Банка России №58-У от 15.10.2025 (опубликовано на сайте ЦБ РФ 17.10.2025, регистрационный номер 03-11-25/00876), с 01.01.2026 вводятся следующие изменения:

В расчёт Показателя включаются просроченные обязательства по кредитам, предоставленным физическим лицам с использованием кредитных карт, даже если они не оформлены как отдельный кредитный договор, а являются частью лимита овердрафта.
Исключается из расчёта просрочка по кредитам, закрытым в течение 30 дней после наступления просрочки, при условии полного погашения задолженности и отсутствия повторных нарушений в течение 12 месяцев.
Методика учёта просрочки по ипотечным кредитам с льготным периодом теперь основывается на фактической дате первого пропущенного платежа после окончания льготы, а не на дате заключения договора.
Документы, регламентирующие изменения:

Указание Банка России №58-У от 15.10.2025 — ссылка на сайт ЦБ
Методические рекомендации №123/2025 — ссылка
Просим обеспечить внесение соответствующих изменений в внутренние системы формирования отчётности и обновление методик расчёта показателей в рамках регуляторного отчёта №110 с 1 января 2026 года.

Ответ на настоящее уведомление не требуется.

С уважением,

Отдел надзора за кредитными организациями
Банк России
Адрес: 107016, г. Москва, ул. Неглинная, д. 12
Тел.: +7 (495) 956-95-95
"""

result = analyze_letter(sample_letter)
print("\n=== PARSED RESULT ===")
print(json.dumps(result, ensure_ascii=False, indent=2))


=== RAW CONTENT ===
<think>
Okay, let's start by analyzing the user's request. They provided a letter from the bank to the presiding officer about changes to the credit indicator calculation method. The task is to extract specific fields into a JSON structure according to the given schema.

First, I need to identify the category. The letter is a notification regarding a regulatory change, so the category should be "notification". 

Next, the deadline_text is mentioned as "1 января 2026 года". The contract number isn't provided, so I'll leave that blank. The laws mentioned are the bank's notice and the methodical recommendations. The emotion here is neutral since it's a regulatory update without any negative or positive sentiment. The tone_of_voice is formal as it's a regulatory notice.

The summary should capture the key points: the change in the calculation method, the deadline, and the reference to the documents. I need to make sure the JSON is strictly in the required format without